# Routes

> Route helpers and utilities for settings endpoints

In [ ]:
#| default_exp routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from pathlib import Path
from fasthtml.common import Div

from cjm_fasthtml_settings.components.forms import create_settings_form_container
from cjm_fasthtml_settings.components.sidebar import create_oob_sidebar_menu
from cjm_fasthtml_settings.components.alerts import create_success_alert, create_error_alert
from cjm_fasthtml_settings.core.html_ids import HtmlIds
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex
from cjm_fasthtml_tailwind.utilities.sizing import min_h
from cjm_fasthtml_tailwind.core.base import combine_classes

## Settings Response Helper

In [ ]:
#| export
def create_settings_response(
    schema: Dict[str, Any],  # Settings schema
    values: Dict[str, Any],  # Configuration values
    save_url: str,  # URL for save action
    reset_url: str,  # URL for reset action
    alert_msg,  # Alert message element
    schemas: Dict[str, Dict[str, Any]],  # All schemas for sidebar
    sidebar_id: str,  # ID for sidebar menu active state
    config_dir: Optional[Path] = None,  # Config directory
    index_route_fn = None,  # Function to generate sidebar route URLs
    menu_section_title: str = "Settings"  # Sidebar section title
) -> Div:  # Div containing form and sidebar with OOB swap
    """Create standardized settings form response with sidebar.

    This consolidates the common pattern of creating a settings form
    with an OOB-swapped sidebar menu.

    Returns:
        Div containing the form and updated sidebar
    """
    return Div(
        create_settings_form_container(
            schema=schema,
            values=values,
            post_url=save_url,
            reset_url=reset_url,
            alert_message=alert_msg
        ),
        create_oob_sidebar_menu(
            schemas=schemas,
            active_schema=sidebar_id,
            config_dir=config_dir,
            index_route_fn=index_route_fn,
            menu_section_title=menu_section_title
        )
    )

## Load Form Content

In [ ]:
#| export
def load_form_content(
    schema: Dict[str, Any],  # Schema to load form for
    config_dir: Optional[Path] = None,  # Config directory
    save_route_fn = None,  # Function to generate save URL
    reset_route_fn = None  # Function to generate reset URL
) -> Div:  # Settings content div
    """Load the settings form content asynchronously.
    
    This is used as the endpoint for async form loading via HTMX.
    """
    from cjm_fasthtml_settings.components.dashboard import render_schema_settings_content
    
    return Div(
        render_schema_settings_content(
            schema=schema,
            config_dir=config_dir,
            save_route_fn=save_route_fn,
            reset_route_fn=reset_route_fn
        ),
        id=HtmlIds.SETTINGS_CONTENT,
        cls=combine_classes(flex(1), min_h(0))
    )

## Save Handler

In [ ]:
#| export
async def handle_save(
    request,  # FastHTML request object
    schema: Dict[str, Any],  # Schema being saved
    schemas: Dict[str, Dict[str, Any]],  # All schemas for sidebar
    schema_id: str,  # ID of the schema
    config_dir: Optional[Path] = None,  # Config directory
    save_route_fn = None,  # Function to generate save URL
    reset_route_fn = None,  # Function to generate reset URL
    index_route_fn = None,  # Function to generate sidebar URLs
    menu_section_title: str = "Settings"  # Sidebar title
):
    """Handle save request for settings.
    
    Converts form data, saves configuration, and returns updated UI.
    """
    from cjm_fasthtml_settings.core.utils import convert_form_data_to_config, save_config
    
    form_data = await request.form()
    config = convert_form_data_to_config(form_data, schema)

    # Generate URLs
    if save_route_fn:
        save_url = save_route_fn(schema_id)
    else:
        save_url = f"/settings/save/{schema_id}"
        
    if reset_route_fn:
        reset_url = reset_route_fn(schema_id)
    else:
        reset_url = f"/settings/reset/{schema_id}"

    # Save configuration
    if save_config(schema_id, config, config_dir):
        return create_settings_response(
            schema=schema,
            values=config,
            save_url=save_url,
            reset_url=reset_url,
            alert_msg=create_success_alert(f"Configuration saved for {schema.get('title')}"),
            schemas=schemas,
            sidebar_id=schema_id,
            config_dir=config_dir,
            index_route_fn=index_route_fn,
            menu_section_title=menu_section_title
        )
    else:
        return create_error_alert(f"Failed to save {schema.get('title')} configuration")

## Reset Handler

In [ ]:
#| export
def handle_reset(
    schema: Dict[str, Any],  # Schema being reset
    schemas: Dict[str, Dict[str, Any]],  # All schemas for sidebar
    schema_id: str,  # ID of the schema
    config_dir: Optional[Path] = None,  # Config directory
    save_route_fn = None,  # Function to generate save URL
    reset_route_fn = None,  # Function to generate reset URL
    index_route_fn = None,  # Function to generate sidebar URLs
    menu_section_title: str = "Settings"  # Sidebar title
):
    """Handle reset request for settings.
    
    Resets configuration to schema defaults and returns updated UI.
    """
    from cjm_fasthtml_settings.core.utils import get_default_values_from_schema
    
    # Use only default values
    values = get_default_values_from_schema(schema)

    # Generate URLs
    if save_route_fn:
        save_url = save_route_fn(schema_id)
    else:
        save_url = f"/settings/save/{schema_id}"
        
    if reset_route_fn:
        reset_url = reset_route_fn(schema_id)
    else:
        reset_url = f"/settings/reset/{schema_id}"

    return create_settings_response(
        schema=schema,
        values=values,
        save_url=save_url,
        reset_url=reset_url,
        alert_msg=create_success_alert("Configuration reset to defaults"),
        schemas=schemas,
        sidebar_id=schema_id,
        config_dir=config_dir,
        index_route_fn=index_route_fn,
        menu_section_title=menu_section_title
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()